In [11]:
import heapq

class Autobot:
    def __init__(self, start, goal, id):
        self.start = start
        self.goal = goal
        self.current_pos = start
        self.id = id
        self.path = []
        self.total_commands = 0
        self.total_time = 0
        self.direction = 0  # 0=up, 1=right, 2=down, 3=left
        self.commands = []
        self.waiting = False  # Initialize waiting attribute

    def forward(self):
        """ Move forward in the direction the bot is facing. """
        if self.direction == 0:  # Up
            return (self.current_pos[0] - 1, self.current_pos[1])
        elif self.direction == 1:  # Right
            return (self.current_pos[0], self.current_pos[1] + 1)
        elif self.direction == 2:  # Down
            return (self.current_pos[0] + 1, self.current_pos[1])
        elif self.direction == 3:  # Left
            return (self.current_pos[0], self.current_pos[1] - 1)

    def reverse(self):
        """ Move backward in the direction opposite to where the bot is facing. """
        if self.direction == 0:  # Up
            return (self.current_pos[0] + 1, self.current_pos[1])
        elif self.direction == 1:  # Right
            return (self.current_pos[0], self.current_pos[1] - 1)
        elif self.direction == 2:  # Down
            return (self.current_pos[0] - 1, self.current_pos[1])
        elif self.direction == 3:  # Left
            return (self.current_pos[0], self.current_pos[1] + 1)

    def left(self):
        """ Turn left (90 degrees). """
        self.direction = (self.direction - 1) % 4
        self.commands.append("Left")
        self.total_commands += 1

    def right(self):
        """ Turn right (90 degrees). """
        self.direction = (self.direction + 1) % 4
        self.commands.append("Right")
        self.total_commands += 1

    def wait(self):
        """ Wait for 1 unit of time. """
        self.waiting = True  # Set waiting state
        self.commands.append("Wait")
        self.total_commands += 1

def is_valid(pos, grid):
    """ Check if the position is within bounds and not an obstacle. """
    x, y = pos
    return 0 <= x < len(grid) and 0 <= y < len(grid[0]) and grid[x][y] == '.'

def heuristic(a, b):
    """ Heuristic for A* (Manhattan distance). """
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star(grid, start, goal):
    """ A* Algorithm for pathfinding. """
    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}

    while open_set:
        _, current = heapq.heappop(open_set)

        if current == goal:
            # Reconstruct path
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path

        for dx, dy in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
            neighbor = (current[0] + dx, current[1] + dy)
            if is_valid(neighbor, grid):
                tentative_g_score = g_score[current] + 1

                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return []  # No path found

def move_bot(bot, grid, positions):
    """ Move bot according to its A* path or wait if it cannot move. """
    bot.waiting = False  # Reset waiting state

    if bot.path:
        next_pos = bot.path[0]  # Get the next position in the path
        current_next_pos = bot.forward()  # Next position based on current direction

        # Determine the needed movement
        if next_pos == current_next_pos:
            # Move forward
            bot.current_pos = next_pos
            bot.path.pop(0)  # Move to the next position
            bot.commands.append("Move Forward")
            bot.total_time += 1
        else:
            # Determine direction to turn
            dx = next_pos[0] - bot.current_pos[0]
            dy = next_pos[1] - bot.current_pos[1]

            if dx == -1 and bot.direction != 0:  # Needs to face up
                while bot.direction != 0:
                    bot.left()  # Turn left until facing up
            elif dx == 1 and bot.direction != 2:  # Needs to face down
                while bot.direction != 2:
                    bot.right()  # Turn right until facing down
            elif dy == 1 and bot.direction != 1:  # Needs to face right
                while bot.direction != 1:
                    bot.right()  # Turn right until facing right
            elif dy == -1 and bot.direction != 3:  # Needs to face left
                while bot.direction != 3:
                    bot.left()  # Turn left until facing left

            # Now attempt to move forward
            if is_valid(bot.forward(), grid) and bot.forward() not in positions.values():
                bot.current_pos = bot.forward()
                bot.path.pop(0)  # Move to the next position
                bot.commands.append("Move Forward")
                bot.total_time += 1
            else:
                bot.wait()  # If can't move forward, wait

def check_deadlock(autobots):
    """ Check for deadlock conditions among autobots. """
    waiting_bots = [bot for bot in autobots if bot.waiting]
    return len(waiting_bots) == len(autobots)  # Deadlock if all are waiting

def simulate_autobots(grid, autobots, max_steps=10000):
    """ Simulate the autobots with collision avoidance and parallel movement. """
    positions = {bot.id: bot.current_pos for bot in autobots}

    for step in range(max_steps):
        moved_positions = {}

        # Parallel move attempt
        for bot in autobots:
            move_bot(bot, grid, positions)
            # If the bot is not waiting, register its new position
            if bot.current_pos not in moved_positions.values():
                moved_positions[bot.id] = bot.current_pos

        positions.update(moved_positions)

        # Check for deadlocks
        if check_deadlock(autobots):
            print("Deadlock detected! Resolving...")
            # Resolve deadlock by letting all waiting bots wait for one time unit
            for bot in autobots:
                if bot.waiting:
                    bot.total_time += 1  # Wait additional time
                    bot.commands.append("Wait")  # Record the command for waiting bots

        if all(bot.current_pos == bot.goal for bot in autobots):
            break  # All bots reached destination

def main():
    # Dynamic grid inputs
    rows = int(input("Enter grid rows: "))
    cols = int(input("Enter grid columns: "))
    grid = [['.' for _ in range(cols)] for _ in range(rows)]

    # Obstacle input
    num_obstacles = int(input("Enter number of obstacles: "))
    for i in range(num_obstacles):
        x, y = map(int, input(f"Enter obstacle {i+1} (x y): ").split())
        grid[x][y] = 'X'

    # Autobot input
    num_bots = int(input("Enter number of autobots: "))
    autobots = []

    for i in range(num_bots):
        sx, sy = map(int, input(f"Enter start position for bot {i+1} (x y): ").split())
        gx, gy = map(int, input(f"Enter end position for bot {i+1} (x y): ").split())
        autobot = Autobot((sx, sy), (gx, gy), i)
        autobot.path = a_star(grid, (sx, sy), (gx, gy))
        autobots.append(autobot)

    # Initial grid display
    print("\nInitial Grid:")
    for r in grid:
        print(' '.join(r))

    # Simulate the autobots
    simulate_autobots(grid, autobots)

    # Output results
    for bot in autobots:
        print(f"\nAutobot {bot.id}:")
        print(f"Total Time Taken: {bot.total_time} units")
        print(f"Total Commands Issued: {bot.total_commands}")
        print("Commands:", ', '.join(bot.commands))

if __name__ == "__main__":
    main()



Enter grid rows: 5
Enter grid columns: 5
Enter number of obstacles: 4
Enter obstacle 1 (x y): 0 3
Enter obstacle 2 (x y): 1 1
Enter obstacle 3 (x y): 2 2
Enter obstacle 4 (x y): 4 1
Enter number of autobots: 2
Enter start position for bot 1 (x y): 0 0
Enter end position for bot 1 (x y): 0 4
Enter start position for bot 2 (x y): 4 0
Enter end position for bot 2 (x y): 4 4

Initial Grid:
. . . X .
. X . . .
. . X . .
. . . . .
. X . . .

Autobot 0:
Total Time Taken: 6 units
Total Commands Issued: 6
Commands: Right, Move Forward, Move Forward, Right, Move Forward, Right, Right, Right, Move Forward, Move Forward, Left, Move Forward

Autobot 1:
Total Time Taken: 6 units
Total Commands Issued: 2
Commands: Move Forward, Right, Move Forward, Move Forward, Move Forward, Move Forward, Right, Move Forward
